In [4]:
import sys
!{sys.executable} -m pip install autogluon.timeseries

  Using cached gluonts-0.15.1-py3-none-any.whl.metadata (9.9 kB)
  Using cached boto3-1.35.56-py3-none-any.whl.metadata (6.7 kB)
  Using cached hyperopt-0.2.7-py2.py3-none-any.whl.metadata (1.7 kB)
  Using cached window_ops-0.0.15-py3-none-any.whl.metadata (6.8 kB)
  Using cached plotly-5.24.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached Markdown-3.7-py3-none-any.whl.metadata (7.0 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-manylinux_2_31_x86_64.whl.metadata (1.1 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached botocore-1.35.56-py3-none-any.whl.metadata (5.7 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached s3transfer-0.10.3-py3-none-any.whl.metadata (1.7 kB)
  Using cached graphviz-0.20.3-py3-none-any.whl.metadata (12 kB)
  Using cached future-1.0.0-py3-none-any.whl.metadata (4.0 kB)
  Using cached py4j-0.10.9

In [1]:
import torch
num_gpus = torch.cuda.device_count()
print("Number of GPUs available:", num_gpus)

for i in range(num_gpus):
    gpu_name = torch.cuda.get_device_name(i)
    print(f"GPU {i}: {gpu_name}")

Number of GPUs available: 1
GPU 0: NVIDIA A30


In [4]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
import pandas as pd
import numpy as np

/home/yl2672496l/Yue/anaconda3/envs/llmtime/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Read data

In [5]:
my_data = pd.read_csv('../GD030A_S.csv')

In [6]:
# Define the recover_timestamp function
def recover_timestamp(data):
    # Combine 'date' and 'time' to form a datetime column
    data['datetime'] = pd.to_datetime(data['date'] + ' ' + data['time'].astype(str) + ':00', format='%Y-%m-%d %H:%M')

    # Set 'datetime' as index
    data = data.set_index('datetime')

    # Create a complete range of timestamps with hourly frequency
    full_time_range = pd.date_range(start=data.index.min(), end=data.index.max(), freq='H')

    # Reindex the data to include all timestamps, filling missing rows with NaN
    data_full = data.reindex(full_time_range)

    return data_full

In [7]:
traffic_full = recover_timestamp(my_data)
traffic_full

,date,time,flow
2019-10-01 00:00:00,2019-10-01,0.0,15.0
2019-10-01 01:00:00,2019-10-01,1.0,9.0
2019-10-01 02:00:00,2019-10-01,2.0,9.0
2019-10-01 03:00:00,2019-10-01,3.0,7.0
2019-10-01 04:00:00,2019-10-01,4.0,9.0
...,...,...,...
2023-09-30 19:00:00,2023-09-30,19.0,129.0
2023-09-30 20:00:00,2023-09-30,20.0,119.0
2023-09-30 21:00:00,2023-09-30,21.0,106.0
2023-09-30 22:00:00,2023-09-30,22.0,88.0


In [25]:
train_set = traffic_full['2022-06-03 00:00:00':'2023-03-31 23:00:00']
valid_set = traffic_full['2023-04-01 00:00:00':'2023-06-30 23:00:00']
test_set = traffic_full['2023-07-01 00:00:00':]
#train_set = traffic_full[:'2022-02-28 23:00:00']
#valid_set = traffic_full['2022-03-01 00:00:00':'2022-12-31 23:00:00']
#test_set = traffic_full['2023-01-01 00:00:00':]
print('Proportion of train_set : {:.4f}'.format(len(train_set)/len(traffic_full)))
print('Proportion of valid_set : {:.4f}'.format(len(valid_set)/len(traffic_full)))
print('Proportion of test_set : {:.4f}'.format(len(test_set)/len(traffic_full)))

Proportion of train_set : 0.2067
Proportion of valid_set : 0.0623
Proportion of test_set : 0.0630


In [9]:
def create_sequences(data, input_length, forecast_horizon):
    """
    Creates sequences for time series data, excluding any sequences containing NaN values.

    Parameters:
    - data: pandas DataFrame containing the data. Must include the 'flow' column.
    - input_length: int, number of past time steps to include in each input sequence.
    - forecast_horizon: int, number of future steps to predict.

    Returns:
    - sequences_df: pandas DataFrame containing all sequences with input lags and output leads,
                    and an 'item_id' column to separate each sequence, including timestamps.
    """
    sequences = []
    item_id = 0  # To assign unique ID to each sequence

    for i in range(input_length, len(data) - forecast_horizon + 1):
        # Extract the input sequence
        X_seq = data.iloc[i - input_length:i]['flow'].values
        X_timestamps = data.iloc[i - input_length:i].index
        # Extract the target sequence
        y_seq = data.iloc[i:i + forecast_horizon]['flow'].values
        y_timestamps = data.iloc[i:i + forecast_horizon].index

        # Check for NaN values in the input sequence and target sequence
        if not np.isnan(X_seq).any() and not np.isnan(y_seq).any():
            # Convert X_seq and y_seq to DataFrames, including their timestamps
            x_df = pd.DataFrame({'target': X_seq, 'timestamp': X_timestamps})
            y_df = pd.DataFrame({'target': y_seq, 'timestamp': y_timestamps})

            # Concatenate X_df and y_df
            seq_df = pd.concat([x_df, y_df], ignore_index=True)

            # Add 'item_id' column
            seq_df['item_id'] = str(item_id)
            item_id += 1  # Increment item_id

            # Append to list
            sequences.append(seq_df)
        else:
            # Optionally, log or count the skipped sequences
            pass  # Simply skip sequences with NaNs

    # Concatenate all sequences into one DataFrame
    sequences_df = pd.concat(sequences)
    sequences_df['item_id'] = sequences_df['item_id'].astype('str')
    sequences_df['target'] = sequences_df['target'].astype('float32')

    return sequences_df

In [10]:
input_lengths = [24 * i for i in range(1, 22)]
prediction_length = 6

In [26]:
from collections import defaultdict
data_dict = defaultdict(dict)

for length in input_lengths:
    print(f"Processing input length: {length}")

    # Create sequences with forecast_horizon=6
    test = create_sequences(test_set, length, forecast_horizon=6)

    # Store in the dictionary
    data_dict[length]['test'] = test

    # Print shapes and ensure no NaNs
    print(f"  test shape: {test.index.max()+1}, {len(test.item_id.unique())}\n")

Processing input length: 24
  test shape: 30, 1719

Processing input length: 48
  test shape: 54, 1604

Processing input length: 72
  test shape: 78, 1508

Processing input length: 96
  test shape: 102, 1412

Processing input length: 120
  test shape: 126, 1316

Processing input length: 144
  test shape: 150, 1220

Processing input length: 168
  test shape: 174, 1124

Processing input length: 192
  test shape: 198, 1028

Processing input length: 216
  test shape: 222, 932

Processing input length: 240
  test shape: 246, 836

Processing input length: 264
  test shape: 270, 740

Processing input length: 288
  test shape: 294, 660

Processing input length: 312
  test shape: 318, 588

Processing input length: 336
  test shape: 342, 516

Processing input length: 360
  test shape: 366, 444

Processing input length: 384
  test shape: 390, 390

Processing input length: 408
  test shape: 414, 342

Processing input length: 432
  test shape: 438, 311

Processing input length: 456
  test shape: 46

# Zero-shot

In [27]:
def chronos_zero_shot(input_lengths, prediction_length, model_type):
    results_dict = defaultdict(dict)

    for length in input_lengths:
        print(f"Processing input length: {length}")

        # prepare the data
        test_data = TimeSeriesDataFrame(data_dict[length]['test'])
        test_data_ready, test_data = test_data.train_test_split(prediction_length)

        predictor = TimeSeriesPredictor(prediction_length=prediction_length).fit(test_data_ready, presets=model_type)
        predictions = predictor.predict(test_data_ready)

        ground_truth_df = test_data.groupby(level='item_id').tail(6)
        predicted_df = predictions[['mean']]

        results_dict[length]['Predicted_Results'] = predicted_df
        results_dict[length]['Gound_Truth'] = ground_truth_df

    return results_dict

In [24]:
torch.cuda.empty_cache()

In [28]:
results_dict1_short = chronos_zero_shot(input_lengths, prediction_length, "chronos_tiny")
results_dict2_short = chronos_zero_shot(input_lengths, prediction_length, "chronos_mini")
results_dict3_short = chronos_zero_shot(input_lengths, prediction_length, "chronos_small")
results_dict4_short = chronos_zero_shot(input_lengths, prediction_length, "chronos_base")
results_dict5_short = chronos_zero_shot(input_lengths, prediction_length, "chronos_large")

Beginning AutoGluon training...
AutoGluon will save models to 'AutogluonModels/ag-20241112_105236'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.9.19
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #18-Ubuntu SMP Fri Oct  4 19:12:19 UTC 2024
CPU Count:          16
GPU Count:          1
Memory Avail:       110.60 GB / 125.71 GB (88.0%)
Disk Space Avail:   391.19 GB / 1003.85 GB (39.0%)
Setting presets to: chronos_tiny

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'hyperparameters': {'Chronos': {'model_path': 'tiny'}},
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 6,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': True,
 'target': 'target',
 'verbosity': 2}

Inferred time series frequency: 'H'
Provided train_data has 41256 rows, 1719 time s

Processing input length: 24


Total runtime: 0.02 s
Best model: Chronos[tiny]
Model not specified in predict, will default to the model with the best validation score: Chronos[tiny]
Beginning AutoGluon training...
AutoGluon will save models to 'AutogluonModels/ag-20241112_105243'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.9.19
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #18-Ubuntu SMP Fri Oct  4 19:12:19 UTC 2024
CPU Count:          16
GPU Count:          1
Memory Avail:       110.62 GB / 125.71 GB (88.0%)
Disk Space Avail:   391.19 GB / 1003.85 GB (39.0%)
Setting presets to: chronos_tiny

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'hyperparameters': {'Chronos': {'model_path': 'tiny'}},
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 6,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 

Processing input length: 48


Models that will be trained: ['Chronos[tiny]']
Training timeseries model Chronos[tiny]. 
	0.00    s     = Training runtime
Training complete. Models trained: ['Chronos[tiny]']
Total runtime: 0.03 s
Best model: Chronos[tiny]
Model not specified in predict, will default to the model with the best validation score: Chronos[tiny]
Beginning AutoGluon training...
AutoGluon will save models to 'AutogluonModels/ag-20241112_105250'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.9.19
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #18-Ubuntu SMP Fri Oct  4 19:12:19 UTC 2024
CPU Count:          16
GPU Count:          1
Memory Avail:       110.58 GB / 125.71 GB (88.0%)
Disk Space Avail:   391.19 GB / 1003.85 GB (39.0%)
Setting presets to: chronos_tiny

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'hyperparameters': {'Chronos': {'model_path': 'tiny'}},
 'known_covariates_names': [],
 'num_val_wi

...


Starting training. Start time is 2024-11-12 11:18:38
Models that will be trained: ['Chronos[large]']
Training timeseries model Chronos[large]. 
	0.00    s     = Training runtime
Training complete. Models trained: ['Chronos[large]']
Total runtime: 0.02 s
Best model: Chronos[large]
Model not specified in predict, will default to the model with the best validation score: Chronos[large]
Beginning AutoGluon training...
AutoGluon will save models to 'AutogluonModels/ag-20241112_111912'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.9.19
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #18-Ubuntu SMP Fri Oct  4 19:12:19 UTC 2024
CPU Count:          16
GPU Count:          1
Memory Avail:       110.59 GB / 125.71 GB (88.0%)
Disk Space Avail:   390.92 GB / 1003.85 GB (38.9%)
Setting presets to: chronos_large

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'hyperparameters': {'Chronos': {'batch_

Processing input length: 480


	This metric's sign has been flipped to adhere to being higher_is_better. The metric score can be multiplied by -1 to get the metric value.

Starting training. Start time is 2024-11-12 11:19:12
Models that will be trained: ['Chronos[large]']
Training timeseries model Chronos[large]. 
	0.00    s     = Training runtime
Training complete. Models trained: ['Chronos[large]']
Total runtime: 0.03 s
Best model: Chronos[large]
Model not specified in predict, will default to the model with the best validation score: Chronos[large]
Beginning AutoGluon training...
AutoGluon will save models to 'AutogluonModels/ag-20241112_111944'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.9.19
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #18-Ubuntu SMP Fri Oct  4 19:12:19 UTC 2024
CPU Count:          16
GPU Count:          1
Memory Avail:       110.51 GB / 125.71 GB (87.9%)
Disk Space Avail:   390.92 GB / 1003.85 GB (38.9%)
Settin

Processing input length: 504


Models that will be trained: ['Chronos[large]']
Training timeseries model Chronos[large]. 
	0.00    s     = Training runtime
Training complete. Models trained: ['Chronos[large]']
Total runtime: 0.02 s
Best model: Chronos[large]
Model not specified in predict, will default to the model with the best validation score: Chronos[large]


In [18]:
results_dict1 = chronos_zero_shot(input_lengths, prediction_length, "chronos_tiny")
results_dict2 = chronos_zero_shot(input_lengths, prediction_length, "chronos_mini")
results_dict3 = chronos_zero_shot(input_lengths, prediction_length, "chronos_small")
results_dict4 = chronos_zero_shot(input_lengths, prediction_length, "chronos_base")
results_dict5 = chronos_zero_shot(input_lengths, prediction_length, "chronos_large")

Beginning AutoGluon training...
AutoGluon will save models to 'AutogluonModels/ag-20241111_130849'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.9.19
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #18-Ubuntu SMP Fri Oct  4 19:12:19 UTC 2024
CPU Count:          16
GPU Count:          1
Memory Avail:       115.19 GB / 125.71 GB (91.6%)
Disk Space Avail:   392.84 GB / 1003.85 GB (39.1%)
Setting presets to: chronos_tiny

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'hyperparameters': {'Chronos': {'model_path': 'tiny'}},
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 6,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': True,
 'target': 'target',
 'verbosity': 2}

Inferred time series frequency: 'H'
Provided train_data has 141960 rows, 5915 time 

Processing input length: 24



Provided data contains following columns:
	target: 'target'

AutoGluon will gauge predictive performance using evaluation metric: 'WQL'
	This metric's sign has been flipped to adhere to being higher_is_better. The metric score can be multiplied by -1 to get the metric value.

Starting training. Start time is 2024-11-11 13:08:49
Models that will be trained: ['Chronos[tiny]']
Training timeseries model Chronos[tiny]. 
	0.00    s     = Training runtime
Training complete. Models trained: ['Chronos[tiny]']
Total runtime: 0.01 s
Best model: Chronos[tiny]
Model not specified in predict, will default to the model with the best validation score: Chronos[tiny]
Beginning AutoGluon training...
AutoGluon will save models to 'AutogluonModels/ag-20241111_130907'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.9.19
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #18-Ubuntu SMP Fri Oct  4 19:12:19 UTC 2024
CPU Count:          

Processing input length: 48


Inferred time series frequency: 'H'
Provided train_data has 274944 rows, 5728 time series. Median time series length is 48 (min=48, max=48). 

Provided data contains following columns:
	target: 'target'

AutoGluon will gauge predictive performance using evaluation metric: 'WQL'
	This metric's sign has been flipped to adhere to being higher_is_better. The metric score can be multiplied by -1 to get the metric value.

Starting training. Start time is 2024-11-11 13:09:07
Models that will be trained: ['Chronos[tiny]']
Training timeseries model Chronos[tiny]. 
	0.00    s     = Training runtime
Training complete. Models trained: ['Chronos[tiny]']
Total runtime: 0.02 s
Best model: Chronos[tiny]
Model not specified in predict, will default to the model with the best validation score: Chronos[tiny]
Beginning AutoGluon training...
AutoGluon will save models to 'AutogluonModels/ag-20241111_130922'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.

...

Beginning AutoGluon training...
AutoGluon will save models to 'AutogluonModels/ag-20241111_152254'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.9.19
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #18-Ubuntu SMP Fri Oct  4 19:12:19 UTC 2024
CPU Count:          16
GPU Count:          1
Memory Avail:       113.50 GB / 125.71 GB (90.3%)
Disk Space Avail:   391.44 GB / 1003.85 GB (39.0%)
Setting presets to: chronos_large

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'hyperparameters': {'Chronos': {'batch_size': 8, 'model_path': 'large'}},
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 6,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': True,
 'target': 'target',
 'verbosity': 2}

Inferred time series frequency: 'H'
Provided train_data has 1369

Processing input length: 480


Beginning AutoGluon training...
AutoGluon will save models to 'AutogluonModels/ag-20241111_152911'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.9.19
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #18-Ubuntu SMP Fri Oct  4 19:12:19 UTC 2024
CPU Count:          16
GPU Count:          1
Memory Avail:       113.56 GB / 125.71 GB (90.3%)
Disk Space Avail:   391.43 GB / 1003.85 GB (39.0%)
Setting presets to: chronos_large

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'hyperparameters': {'Chronos': {'batch_size': 8, 'model_path': 'large'}},
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 6,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': True,
 'target': 'target',
 'verbosity': 2}

Inferred time series frequency: 'H'
Provided train_data has 1383

Processing input length: 504


Beginning AutoGluon training...
AutoGluon will save models to 'AutogluonModels/ag-20241111_153527'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.9.19
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #18-Ubuntu SMP Fri Oct  4 19:12:19 UTC 2024
CPU Count:          16
GPU Count:          1
Memory Avail:       113.41 GB / 125.71 GB (90.2%)
Disk Space Avail:   391.41 GB / 1003.85 GB (39.0%)
Setting presets to: chronos_large

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'hyperparameters': {'Chronos': {'batch_size': 8, 'model_path': 'large'}},
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 6,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': True,
 'target': 'target',
 'verbosity': 2}

Inferred time series frequency: 'H'
Provided train_data has 1401

In [14]:
def cal_metrics(df_true, df_pred):
    df_merged = pd.merge(df_true, df_pred, on=['item_id', 'timestamp'], how='inner')

    # Step 3: Assign prediction step numbers (1 to 6) within each 'item_id'
    df_merged['step'] = df_merged.groupby('item_id').cumcount() + 1

    # Step 4: Calculate error metrics
    df_merged['error'] = df_merged['mean'] - df_merged['target']
    df_merged['abs_error'] = df_merged['error'].abs()
    df_merged['squared_error'] = df_merged['error'] ** 2

    # Handle division by zero in MAPE calculation by adding a very small number (epsilon)
    epsilon = 1e-10
    df_merged['abs_percentage_error'] = df_merged['abs_error'] / (df_merged['target'].abs() + epsilon)

    # Step 5: Calculate overall metrics
    overall_mse = df_merged['squared_error'].mean()
    overall_rmse = np.sqrt(overall_mse)
    overall_mae = df_merged['abs_error'].mean()
    overall_mape = df_merged['abs_percentage_error'].mean() * 100

    print("Overall Evaluation Metrics:")
    print(f"MSE: {overall_mse:.4f}")
    print(f"RMSE: {overall_rmse:.4f}")
    print(f"MAE: {overall_mae:.4f}")
    print(f"MAPE: {overall_mape:.2f}%")

    # Step 6: Calculate metrics for each prediction step
    metrics_by_step = df_merged.groupby('step').agg({
      'squared_error': 'mean',
      'abs_error': 'mean',
      'abs_percentage_error': 'mean'
    }).reset_index()

    metrics_by_step['MSE'] = metrics_by_step['squared_error']
    metrics_by_step['RMSE'] = np.sqrt(metrics_by_step['squared_error'])
    metrics_by_step['MAE'] = metrics_by_step['abs_error']
    metrics_by_step['MAPE'] = metrics_by_step['abs_percentage_error'] * 100

    # Select relevant columns and round for better readability
    metrics_by_step = metrics_by_step[['step', 'MSE', 'RMSE', 'MAE', 'MAPE']]

    return metrics_by_step

In [15]:
def metrics_results(results_dict):
    for length in input_lengths:
        print(f"Processing input length: {length}")
    
        predict = results_dict[length]['Predicted_Results']
        truth = results_dict[length]['Gound_Truth']
        evaluation_df = cal_metrics(predict, truth)
        display(evaluation_df)

In [29]:
metrics_results(results_dict1_short)

Processing input length: 24
Overall Evaluation Metrics:
MSE: 1168.9452
RMSE: 34.1898
MAE: 26.1718
MAPE: 23.07%


,step,MSE,RMSE,MAE,MAPE
0,1,714.890348,26.737433,20.520652,17.792879
1,2,1031.789429,32.121479,24.310202,21.166429
2,3,1186.666493,34.448026,26.428506,22.870633
3,4,1276.404827,35.726808,27.531757,24.288234
4,5,1357.730214,36.847391,28.378794,25.212371
5,6,1446.189613,38.028800,29.861015,27.076341


Processing input length: 48
Overall Evaluation Metrics:
MSE: 909.5197
RMSE: 30.1582
MAE: 22.7534
MAPE: 19.67%


,step,MSE,RMSE,MAE,MAPE
0,1,610.971988,24.717848,18.873733,16.524192
1,2,796.393599,28.220446,21.340644,18.632875
2,3,885.612524,29.759243,22.782911,19.795225
3,4,973.045340,31.193675,23.590603,20.345880
4,5,1062.585968,32.597331,24.461813,20.960492
5,6,1128.509033,33.593289,25.470463,21.790994


Processing input length: 72
Overall Evaluation Metrics:
MSE: 861.9512
RMSE: 29.3590
MAE: 21.8325
MAPE: 18.43%


...

,step,MSE,RMSE,MAE,MAPE
0,1,316.308125,17.785053,13.981094,11.216939
1,2,316.422467,17.788268,13.605381,11.025731
2,3,298.152455,17.267092,12.881871,10.564327
3,4,325.711998,18.047493,13.590884,10.964042
4,5,338.903837,18.409341,13.870568,11.094483
5,6,361.357199,19.009398,14.335540,11.558934


Processing input length: 480
Overall Evaluation Metrics:
MSE: 319.4392
RMSE: 17.8729
MAE: 13.4397
MAPE: 10.91%


,step,MSE,RMSE,MAE,MAPE
0,1,321.032941,17.917392,13.930256,11.269119
1,2,310.031548,17.607713,13.396488,10.866031
2,3,299.339336,17.301426,12.858683,10.577925
3,4,320.329691,17.897757,13.406083,10.842008
4,5,319.187362,17.865815,13.366421,10.744807
5,6,346.714439,18.620270,13.680223,11.135733


Processing input length: 504
Overall Evaluation Metrics:
MSE: 340.3718
RMSE: 18.4492
MAE: 13.8633
MAPE: 11.34%


,step,MSE,RMSE,MAE,MAPE
0,1,333.984641,18.275247,14.171563,11.553421
1,2,332.445596,18.233091,13.864454,11.300753
2,3,316.401962,17.787691,12.988515,10.628358
3,4,333.961675,18.274618,13.539948,11.017957
4,5,356.577026,18.883247,14.283212,11.667627
5,6,368.860051,19.205730,14.331881,11.852748


In [30]:
metrics_results(results_dict2_short)

Processing input length: 24
Overall Evaluation Metrics:
MSE: 1083.9306
RMSE: 32.9231
MAE: 25.1285
MAPE: 22.47%


,step,MSE,RMSE,MAE,MAPE
0,1,666.408003,25.814879,19.833516,17.454014
1,2,963.079026,31.033515,23.698025,20.897891
2,3,1066.652026,32.659639,24.892348,21.949420
3,4,1179.393946,34.342305,26.238794,23.642015
4,5,1277.179398,35.737647,27.482053,24.635297
5,6,1350.871335,36.754202,28.626548,26.220421


Processing input length: 48
Overall Evaluation Metrics:
MSE: 940.6464
RMSE: 30.6700
MAE: 22.9150
MAPE: 19.64%


,step,MSE,RMSE,MAE,MAPE
0,1,608.578651,24.669387,18.664348,16.425684
1,2,808.655481,28.436868,21.460797,18.723472
2,3,919.285907,30.319728,22.914628,19.884247
3,4,1017.448276,31.897465,23.945660,20.270631
4,5,1114.463754,33.383585,24.864777,20.894748
5,6,1175.446188,34.284781,25.639931,21.657716


Processing input length: 72
Overall Evaluation Metrics:
MSE: 847.0084
RMSE: 29.1034
MAE: 21.5196
MAPE: 18.35%


...

,step,MSE,RMSE,MAE,MAPE
0,1,273.081158,16.525167,13.103428,10.608199
1,2,264.956671,16.277490,12.589572,10.154002
2,3,253.887660,15.933853,12.249577,9.871595
3,4,278.022619,16.674010,12.980161,10.530926
4,5,286.888655,16.937788,13.099566,10.597368
5,6,293.987366,17.146060,13.262892,10.734302


Processing input length: 480
Overall Evaluation Metrics:
MSE: 280.6930
RMSE: 16.7539
MAE: 12.8370
MAPE: 10.34%


,step,MSE,RMSE,MAE,MAPE
0,1,280.861203,16.758914,13.111059,10.629003
1,2,277.910657,16.670653,12.609430,10.190557
2,3,269.299038,16.410333,12.299102,9.804942
3,4,278.131966,16.677289,12.957338,10.284342
4,5,287.138896,16.945173,12.867434,10.382715
5,6,290.816158,17.053333,13.177857,10.726760


Processing input length: 504
Overall Evaluation Metrics:
MSE: 280.4587
RMSE: 16.7469
MAE: 12.8658
MAPE: 10.46%


,step,MSE,RMSE,MAE,MAPE
0,1,271.896375,16.489281,12.871015,10.596164
1,2,273.882265,16.549389,12.700197,10.324306
2,3,268.677536,16.391386,12.501556,10.167297
3,4,285.673891,16.901890,13.071450,10.478037
4,5,283.732539,16.844362,12.951888,10.567998
5,6,298.889300,17.288415,13.098832,10.646554


In [31]:
metrics_results(results_dict3_short)

Processing input length: 24
Overall Evaluation Metrics:
MSE: 1097.2296
RMSE: 33.1245
MAE: 25.4027
MAPE: 22.90%


,step,MSE,RMSE,MAE,MAPE
0,1,675.277154,25.986095,20.011550,17.662815
1,2,964.796019,31.061166,23.783753,21.205044
2,3,1081.785711,32.890511,25.097126,22.614972
3,4,1178.201102,34.324934,26.384937,23.950258
4,5,1297.876549,36.026054,27.882387,25.000811
5,6,1385.441165,37.221515,29.256567,26.953524


Processing input length: 48
Overall Evaluation Metrics:
MSE: 932.9414
RMSE: 30.5441
MAE: 22.9315
MAPE: 20.11%


,step,MSE,RMSE,MAE,MAPE
0,1,593.358893,24.358959,18.513222,16.442749
1,2,790.962244,28.124051,21.213979,18.831206
2,3,913.237907,30.219826,22.951652,20.295063
3,4,1020.707296,31.948510,24.099367,20.952536
4,5,1101.358312,33.186719,24.971644,21.659055
5,6,1178.023835,34.322352,25.839199,22.485889


Processing input length: 72
Overall Evaluation Metrics:
MSE: 841.8233
RMSE: 29.0142
MAE: 21.7317
MAPE: 18.69%


...

,step,MSE,RMSE,MAE,MAPE
0,1,170.415348,13.054323,9.989217,8.308781
1,2,159.865110,12.643778,9.446443,7.749249
2,3,153.416809,12.386154,9.162207,7.395183
3,4,169.504258,13.019380,9.530498,7.698227
4,5,170.426015,13.054732,9.726436,7.794811
5,6,171.673275,13.102415,9.894271,7.855677


Processing input length: 480
Overall Evaluation Metrics:
MSE: 160.1867
RMSE: 12.6565
MAE: 9.4891
MAPE: 7.56%


,step,MSE,RMSE,MAE,MAPE
0,1,177.257962,13.313826,10.277758,8.391616
1,2,162.594429,12.751252,9.515027,7.632839
2,3,152.500430,12.349106,9.111605,7.284854
3,4,151.791587,12.320373,9.110045,7.148770
4,5,157.767130,12.560539,9.426947,7.397116
5,6,159.208663,12.617792,9.493123,7.492255


Processing input length: 504
Overall Evaluation Metrics:
MSE: 163.8069
RMSE: 12.7987
MAE: 9.5891
MAPE: 7.65%


,step,MSE,RMSE,MAE,MAPE
0,1,177.038271,13.305573,10.133009,8.366198
1,2,158.192454,12.577458,9.385971,7.575245
2,3,161.158100,12.694806,9.474853,7.475754
3,4,169.770723,13.029609,9.709653,7.664982
4,5,153.429935,12.386684,9.315152,7.298487
5,6,163.251842,12.777004,9.515663,7.497449


In [32]:
metrics_results(results_dict4_short)

Processing input length: 24
Overall Evaluation Metrics:
MSE: 1135.5615
RMSE: 33.6981
MAE: 25.8533
MAPE: 23.29%


,step,MSE,RMSE,MAE,MAPE
0,1,652.017019,25.534624,19.720410,17.687818
1,2,951.038921,30.838919,23.712946,21.133943
2,3,1074.769435,32.783676,25.259594,22.562724
3,4,1233.258960,35.117787,27.076372,24.453975
4,5,1390.472291,37.289037,29.120097,26.120897
5,6,1511.812227,38.882030,30.230639,27.779798


Processing input length: 48
Overall Evaluation Metrics:
MSE: 914.7634
RMSE: 30.2451
MAE: 22.5318
MAPE: 19.89%


,step,MSE,RMSE,MAE,MAPE
0,1,567.615681,23.824686,18.261254,16.400853
1,2,787.040234,28.054237,21.067014,18.686579
2,3,906.906874,30.114895,22.652932,20.092282
3,4,1014.999397,31.859055,23.850907,20.817292
4,5,1069.670148,32.705812,24.295115,21.295966
5,6,1142.347851,33.798637,25.063311,22.037370


Processing input length: 72
Overall Evaluation Metrics:
MSE: 827.3326
RMSE: 28.7634
MAE: 21.5385
MAPE: 19.00%


...

,step,MSE,RMSE,MAE,MAPE
0,1,133.650560,11.560734,8.778246,7.079208
1,2,123.485943,11.112423,8.155979,6.478880
2,3,126.948079,11.267124,8.390760,6.727664
3,4,133.281429,11.544758,8.782051,7.073881
4,5,129.299765,11.371005,8.607084,6.887301
5,6,135.601103,11.644789,8.811066,6.942660


Processing input length: 480
Overall Evaluation Metrics:
MSE: 131.6085
RMSE: 11.4721
MAE: 8.6741
MAPE: 6.91%


,step,MSE,RMSE,MAE,MAPE
0,1,132.614888,11.515854,8.749441,6.938700
1,2,131.756283,11.478514,8.472026,6.795154
2,3,131.291237,11.458239,8.707890,7.011922
3,4,127.452629,11.289492,8.613460,6.841759
4,5,135.467239,11.639039,8.811917,6.948607
5,6,131.068511,11.448516,8.689701,6.896460


Processing input length: 504
Overall Evaluation Metrics:
MSE: 134.8077
RMSE: 11.6107
MAE: 8.6647
MAPE: 6.89%


,step,MSE,RMSE,MAE,MAPE
0,1,128.236604,11.324160,8.610971,7.005590
1,2,132.689959,11.519113,8.411443,6.729203
2,3,131.405326,11.463216,8.591873,6.784453
3,4,141.477761,11.894442,8.965947,7.067385
4,5,139.079191,11.793184,8.699266,6.892550
5,6,135.957467,11.660080,8.708901,6.869425


In [33]:
metrics_results(results_dict5_short)

Processing input length: 24
Overall Evaluation Metrics:
MSE: 1134.9435
RMSE: 33.6889
MAE: 25.9380
MAPE: 23.35%


,step,MSE,RMSE,MAE,MAPE
0,1,663.798255,25.764283,19.821895,17.344716
1,2,964.817636,31.061514,24.010066,21.216768
2,3,1104.803753,33.238588,25.556927,22.810578
3,4,1230.193689,35.074117,27.238484,24.721376
4,5,1370.866781,37.025218,28.980470,26.203193
5,6,1475.180846,38.408083,30.019980,27.791765


Processing input length: 48
Overall Evaluation Metrics:
MSE: 905.1468
RMSE: 30.0857
MAE: 22.4692
MAPE: 19.93%


,step,MSE,RMSE,MAE,MAPE
0,1,580.109864,24.085470,18.450476,16.501662
1,2,779.973392,27.928004,20.947538,18.761944
2,3,901.541332,30.025678,22.682099,20.395878
3,4,986.557203,31.409508,23.693894,20.829563
4,5,1063.300286,32.608286,24.265672,21.188080
5,6,1119.398568,33.457414,24.775693,21.899075


Processing input length: 72
Overall Evaluation Metrics:
MSE: 786.0261
RMSE: 28.0362
MAE: 21.1135
MAPE: 18.35%


...

,step,MSE,RMSE,MAE,MAPE
0,1,112.643512,10.613365,7.989826,6.271104
1,2,126.185911,11.233250,8.324241,6.569829
2,3,115.317446,10.738596,7.822439,6.256136
3,4,126.532408,11.248662,8.173078,6.546453
4,5,128.209276,11.322954,8.344249,6.607726
5,6,129.504827,11.380019,8.305833,6.598610


Processing input length: 480
Overall Evaluation Metrics:
MSE: 124.9269
RMSE: 11.1771
MAE: 8.1488
MAPE: 6.44%


,step,MSE,RMSE,MAE,MAPE
0,1,118.531720,10.887227,8.227408,6.569857
1,2,124.948667,11.178044,8.112379,6.453574
2,3,115.857790,10.763726,7.803436,6.215820
3,4,124.255089,11.146977,8.067185,6.315371
4,5,133.949805,11.573669,8.323854,6.549983
5,6,132.018558,11.489933,8.358613,6.545076


Processing input length: 504
Overall Evaluation Metrics:
MSE: 127.9418
RMSE: 11.3111
MAE: 8.2974
MAPE: 6.48%


,step,MSE,RMSE,MAE,MAPE
0,1,118.585872,10.889714,8.305800,6.464279
1,2,125.560417,11.205374,8.162517,6.345661
2,3,127.372392,11.285938,8.248281,6.470159
3,4,133.417823,11.550663,8.372309,6.537146
4,5,130.823030,11.437790,8.363706,6.549148
5,6,131.891063,11.484383,8.331810,6.488802


In [19]:
metrics_results(results_dict1)

Processing input length: 24
Overall Evaluation Metrics:
MSE: 2244.3584
RMSE: 47.3747
MAE: 32.5315
MAPE: 34.92%


,step,MSE,RMSE,MAE,MAPE
0,1,1298.925490,36.040609,24.836219,25.205852
1,2,1872.780967,43.275639,29.960267,30.572911
2,3,2218.228326,47.098071,32.453996,33.784616
3,4,2444.599725,49.442894,34.219329,36.914544
4,5,2688.532095,51.851057,36.071117,40.515557
5,6,2943.084039,54.250199,37.648348,42.549825


Processing input length: 48
Overall Evaluation Metrics:
MSE: 1983.0282
RMSE: 44.5312
MAE: 29.4247
MAPE: 31.65%


,step,MSE,RMSE,MAE,MAPE
0,1,1228.491676,35.049846,23.609824,24.053533
1,2,1692.785605,41.143476,27.313469,28.002762
2,3,1962.360583,44.298539,29.500414,31.049345
3,4,2160.769397,46.484077,30.917706,33.515965
4,5,2348.487830,48.461199,32.146311,36.051962
5,6,2505.274328,50.052715,33.060712,37.223322


Processing input length: 72
Overall Evaluation Metrics:
MSE: 1942.6707
RMSE: 44.0757
MAE: 28.8227
MAPE: 30.52%


...

,step,MSE,RMSE,MAE,MAPE
0,1,1264.968015,35.566389,23.156213,24.350749
1,2,1652.333501,40.648905,25.504075,26.445103
2,3,1858.085688,43.105518,26.743992,27.989781
3,4,2002.117203,44.745024,27.900457,29.046560
4,5,2204.218185,46.949102,28.776862,29.816418
5,6,2385.040383,48.836875,29.482630,30.216758


Processing input length: 480
Overall Evaluation Metrics:
MSE: 1842.4510
RMSE: 42.9238
MAE: 26.7291
MAPE: 28.31%


,step,MSE,RMSE,MAE,MAPE
0,1,1257.523947,35.461584,23.087470,24.961110
1,2,1616.415918,40.204675,25.323669,26.610397
2,3,1772.095518,42.096265,26.394914,27.969531
3,4,1933.834970,43.975391,27.699889,29.379341
4,5,2137.631901,46.234531,28.541039,30.215299
5,6,2337.203824,48.344636,29.327807,30.705733


Processing input length: 504
Overall Evaluation Metrics:
MSE: 1791.0273
RMSE: 42.3205
MAE: 26.3427
MAPE: 28.33%


,step,MSE,RMSE,MAE,MAPE
0,1,1196.260967,34.587006,22.731666,24.706446
1,2,1549.590049,39.364833,24.911869,26.876933
2,3,1743.809106,41.758940,26.195813,28.279447
3,4,1905.411670,43.651021,27.384830,29.438044
4,5,2073.165488,45.532027,28.004695,30.075250
5,6,2277.926480,47.727628,28.827535,30.600878


In [20]:
metrics_results(results_dict2)

Processing input length: 24
Overall Evaluation Metrics:
MSE: 2086.5086
RMSE: 45.6783
MAE: 31.2590
MAPE: 34.09%


,step,MSE,RMSE,MAE,MAPE
0,1,1215.011888,34.857021,24.124751,24.912342
1,2,1747.403118,41.801951,28.886207,30.104729
2,3,2045.393675,45.226029,31.163016,33.148319
3,4,2267.964678,47.623153,32.855978,36.140566
4,5,2516.556947,50.165296,34.569557,39.169995
5,6,2726.721417,52.218018,35.954516,41.038699


Processing input length: 48
Overall Evaluation Metrics:
MSE: 1980.3288
RMSE: 44.5009
MAE: 29.4049
MAPE: 32.03%


,step,MSE,RMSE,MAE,MAPE
0,1,1214.392470,34.848134,23.416480,24.400909
1,2,1682.410536,41.017198,27.237153,28.599080
2,3,1957.102909,44.239156,29.455704,31.690442
3,4,2145.946666,46.324364,30.948756,34.080024
4,5,2352.757886,48.505236,32.173798,36.168708
5,6,2529.362245,50.292765,33.197763,37.270662


Processing input length: 72
Overall Evaluation Metrics:
MSE: 1923.9962
RMSE: 43.8634
MAE: 28.6557
MAPE: 31.16%


...

,step,MSE,RMSE,MAE,MAPE
0,1,1180.975390,34.365322,22.348165,23.473719
1,2,1580.355787,39.753689,24.621761,25.446610
2,3,1774.711373,42.127323,25.865467,26.664426
3,4,1922.643078,43.847954,27.001984,27.666409
4,5,2108.230054,45.915466,27.695989,28.354356
5,6,2300.488194,47.963405,28.517455,28.967681


Processing input length: 480
Overall Evaluation Metrics:
MSE: 1744.0530
RMSE: 41.7619
MAE: 25.8040
MAPE: 27.11%


,step,MSE,RMSE,MAE,MAPE
0,1,1169.365802,34.195991,22.326325,24.000803
1,2,1537.969576,39.216955,24.531668,25.567937
2,3,1683.447513,41.029837,25.525544,26.794544
3,4,1851.575251,43.029934,26.866046,28.191160
4,5,2029.944806,45.054909,27.479855,28.885199
5,6,2192.014907,46.818959,28.094608,29.205075


Processing input length: 504
Overall Evaluation Metrics:
MSE: 1671.3130
RMSE: 40.8817
MAE: 25.2216
MAPE: 26.93%


,step,MSE,RMSE,MAE,MAPE
0,1,1122.489860,33.503580,21.924853,23.942785
1,2,1471.060913,38.354412,24.042875,25.981146
2,3,1623.280929,40.289961,25.059078,26.778167
3,4,1774.265417,42.122030,26.168286,27.658091
4,5,1937.817169,44.020645,26.647516,28.296992
5,6,2098.964011,45.814452,27.487244,28.940789


In [21]:
metrics_results(results_dict3)

Processing input length: 24
Overall Evaluation Metrics:
MSE: 2100.1813
RMSE: 45.8277
MAE: 31.4836
MAPE: 34.76%


,step,MSE,RMSE,MAE,MAPE
0,1,1209.548594,34.778565,24.128148,24.844088
1,2,1770.291427,42.074831,29.180957,30.602435
2,3,2067.106848,45.465447,31.404846,33.905339
3,4,2281.604012,47.766139,32.992781,36.934805
4,5,2526.081063,50.260134,34.921466,40.095420
5,6,2746.455690,52.406638,36.273603,42.194477


Processing input length: 48
Overall Evaluation Metrics:
MSE: 1958.9268
RMSE: 44.2598
MAE: 29.3186
MAPE: 32.27%


,step,MSE,RMSE,MAE,MAPE
0,1,1197.716321,34.608038,23.281632,24.252554
1,2,1658.924943,40.729902,27.028421,28.482915
2,3,1930.224810,43.934324,29.271749,31.818382
3,4,2135.008216,46.206149,30.920144,34.445606
4,5,2321.927148,48.186379,32.212275,36.656681
5,6,2509.759362,50.097499,33.197190,37.960949


Processing input length: 72
Overall Evaluation Metrics:
MSE: 1911.3553
RMSE: 43.7190
MAE: 28.4486
MAPE: 30.39%


...

,step,MSE,RMSE,MAE,MAPE
0,1,1168.046474,34.176695,22.069852,23.261198
1,2,1549.483013,39.363473,24.193759,24.956476
2,3,1754.301085,41.884378,25.462362,26.384552
3,4,1912.601312,43.733298,26.570398,27.395280
4,5,2118.022435,46.021978,27.497114,28.394005
5,6,2299.536871,47.953487,28.138350,28.698238


Processing input length: 480
Overall Evaluation Metrics:
MSE: 1745.7989
RMSE: 41.7828
MAE: 25.4222
MAPE: 26.84%


,step,MSE,RMSE,MAE,MAPE
0,1,1160.676038,34.068696,21.994205,23.861059
1,2,1518.239437,38.964592,24.002245,25.093460
2,3,1687.711178,41.081762,25.196968,26.458411
3,4,1843.763561,42.939068,26.370952,27.795092
4,5,2051.960266,45.298568,27.169250,28.924441
5,6,2212.443076,47.036614,27.799669,28.880422


Processing input length: 504
Overall Evaluation Metrics:
MSE: 1648.3853
RMSE: 40.6003
MAE: 24.7163
MAPE: 26.38%


,step,MSE,RMSE,MAE,MAPE
0,1,1102.670535,33.206483,21.487262,23.456025
1,2,1429.250149,37.805425,23.351578,24.998906
2,3,1608.205293,40.102435,24.649119,26.207913
3,4,1758.728632,41.937199,25.568884,27.042962
4,5,1920.163911,43.819675,26.228371,28.035598
5,6,2071.293381,45.511464,27.012537,28.535548


In [22]:
metrics_results(results_dict4)

Processing input length: 24
Overall Evaluation Metrics:
MSE: 2115.9591
RMSE: 45.9996
MAE: 31.5710
MAPE: 34.65%


,step,MSE,RMSE,MAE,MAPE
0,1,1180.119803,34.352872,23.687548,24.698520
1,2,1745.715069,41.781755,28.866582,30.266106
2,3,2056.349401,45.346989,31.245471,33.424009
3,4,2313.074124,48.094429,33.375681,36.895048
4,5,2584.344184,50.836445,35.457268,40.226419
5,6,2816.152126,53.067430,36.793250,42.407413


Processing input length: 48
Overall Evaluation Metrics:
MSE: 1939.9509
RMSE: 44.0449
MAE: 28.9413
MAPE: 31.77%


,step,MSE,RMSE,MAE,MAPE
0,1,1157.103953,34.016231,22.950045,24.016099
1,2,1637.420690,40.465055,26.761298,28.245351
2,3,1908.747192,43.689211,28.903043,31.015182
3,4,2128.249114,46.132950,30.594914,33.803685
4,5,2318.297459,48.148702,31.767176,36.207473
5,6,2489.886766,49.898765,32.671605,37.325170


Processing input length: 72
Overall Evaluation Metrics:
MSE: 1875.2066
RMSE: 43.3037
MAE: 28.0393
MAPE: 30.67%


...

,step,MSE,RMSE,MAE,MAPE
0,1,1146.812796,33.864625,21.670719,22.634434
1,2,1533.192978,39.156008,23.858970,24.375813
2,3,1692.707257,41.142524,24.766423,25.463015
3,4,1826.479555,42.737332,25.697152,26.177269
4,5,2010.031792,44.833378,26.555099,27.111844
5,6,2165.450025,46.534396,27.110616,27.219304


Processing input length: 480
Overall Evaluation Metrics:
MSE: 1687.6081
RMSE: 41.0805
MAE: 24.8054
MAPE: 25.72%


,step,MSE,RMSE,MAE,MAPE
0,1,1137.092306,33.720799,21.576972,22.970767
1,2,1496.893989,38.689714,23.695764,24.490660
2,3,1628.546027,40.355248,24.490669,25.324960
3,4,1780.303615,42.193644,25.610135,26.426948
4,5,1972.522436,44.413089,26.421701,27.394154
5,6,2110.290378,45.937897,27.037189,27.718225


Processing input length: 504
Overall Evaluation Metrics:
MSE: 1597.9883
RMSE: 39.9748
MAE: 24.1986
MAPE: 25.55%


,step,MSE,RMSE,MAE,MAPE
0,1,1067.653400,32.674966,21.023029,22.613987
1,2,1421.048480,37.696797,23.094770,24.385565
2,3,1555.775246,39.443317,24.033140,25.403969
3,4,1688.926596,41.096552,25.023989,26.317545
4,5,1848.295302,42.991805,25.610246,26.971840
5,6,2006.231009,44.790970,26.406622,27.621844


In [23]:
metrics_results(results_dict5)

Processing input length: 24
Overall Evaluation Metrics:
MSE: 2111.9273
RMSE: 45.9557
MAE: 31.4235
MAPE: 34.25%


,step,MSE,RMSE,MAE,MAPE
0,1,1185.527916,34.431496,23.699455,24.324120
1,2,1741.626972,41.732805,28.715293,29.754903
2,3,2056.510643,45.348767,31.278611,33.362028
3,4,2335.784180,48.329951,33.300667,36.657644
4,5,2575.258301,50.747003,35.126678,39.892904
5,6,2776.855615,52.695879,36.420494,41.523559


Processing input length: 48
Overall Evaluation Metrics:
MSE: 1946.1900
RMSE: 44.1156
MAE: 28.9377
MAPE: 31.33%


,step,MSE,RMSE,MAE,MAPE
0,1,1179.551990,34.344606,23.059380,23.927273
1,2,1634.105544,40.424071,26.671603,27.949269
2,3,1923.144193,43.853668,28.990240,30.793178
3,4,2141.896683,46.280630,30.552960,33.279199
4,5,2312.725696,48.090807,31.774612,35.430869
5,6,2485.716056,49.856956,32.577418,36.596839


Processing input length: 72
Overall Evaluation Metrics:
MSE: 1873.4104
RMSE: 43.2829
MAE: 28.0132
MAPE: 30.45%


...

,step,MSE,RMSE,MAE,MAPE
0,1,1114.466506,33.383626,21.417539,22.440802
1,2,1478.846775,38.455777,23.454606,23.841001
2,3,1657.790825,40.715978,24.436786,24.793169
3,4,1806.746333,42.505839,25.547082,25.899656
4,5,2012.859531,44.864903,26.355028,26.644384
5,6,2164.520357,46.524406,27.137950,27.006422


Processing input length: 480
Overall Evaluation Metrics:
MSE: 1644.3676
RMSE: 40.5508
MAE: 24.4520
MAPE: 25.20%


,step,MSE,RMSE,MAE,MAPE
0,1,1108.494601,33.294063,21.289274,22.531397
1,2,1450.233814,38.081936,23.395243,24.344996
2,3,1571.857737,39.646661,24.015932,24.750612
3,4,1735.293801,41.656858,25.361793,25.891548
4,5,1932.992703,43.965813,26.004545,26.615075
5,6,2067.333149,45.467935,26.645087,27.074050


Processing input length: 504
Overall Evaluation Metrics:
MSE: 1575.1554
RMSE: 39.6882
MAE: 23.9211
MAPE: 25.04%


,step,MSE,RMSE,MAE,MAPE
0,1,1054.598508,32.474582,20.949148,22.584264
1,2,1377.405310,37.113411,22.798492,24.223898
2,3,1524.389729,39.043434,23.694422,24.978235
3,4,1673.003816,40.902369,24.727979,25.424359
4,5,1834.543857,42.831575,25.285924,26.265561
5,6,1986.991218,44.575680,26.070774,26.741689
